In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
import numpy as np

In [6]:
from keras.models import model_from_json
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from pickle import load

In [35]:
def load_model(location, tag):
    json_file = open(location + f'models/{tag}/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(location + f'models/{tag}/model.h5')
    scaler = load(open(location + f'models/{tag}/scaler.pkl', 'rb'))
    return scaler, loaded_model

In [36]:
import os
os.getcwd()

'/uscms_data/d3/srosenzw/workarea/higgs/sixb_analysis/CMSSW_10_2_18/src/sixb/6jet_classifier/models'

In [37]:
scaler, model = load_model(location='../', tag='20210828_multiclass_nsignal')

In [38]:
scaler_min = np.asarray(scaler.data_min_)

In [39]:
scaler_max = np.asarray(scaler.data_max_)

In [40]:
min_max = np.row_stack((scaler_min, scaler_max))

In [41]:
import os
os.getcwd()

'/uscms_data/d3/srosenzw/workarea/higgs/sixb_analysis/CMSSW_10_2_18/src/sixb/6jet_classifier/models'

In [42]:
np.savetxt("scaler2b.csv", min_max, delimiter=',')

In [43]:
import tensorflow.python.keras.backend as K
sess = K.get_session()

In [44]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [45]:
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
# tf.train.write_graph(frozen_graph, "2b_models", "my_model2b.pb", as_text=False)

In [46]:
[out.op.name for out in model.outputs]

['dense_2_1/Softmax']

In [47]:
features = np.asarray([120.153,  114.914,  75.994,  75.099,  61.896,  61.564, -0.525,
       -0.278, -0.992,  0.311, -1.057, -1.844, -1.140,  1.660,  0.878,
        2.717,  3.042, -1.256,  0.778,  0.700,  0.401,  0.996,  0.003,
        0.996,  108.143,  124.577,  80.956,  79.714,  67.812,  42.099])

In [48]:
type(scaler.min_)

numpy.ndarray

In [49]:
features = np.asarray(numbers.split(' '), dtype=float)

In [50]:
type(scaler.data_min_)

numpy.ndarray

In [51]:
features = features.reshape(1,30)
features = scaler.transform(features)

In [52]:
features

array([[0.06963656, 0.06634983, 0.03817676, 0.03091153, 0.02774934,
        0.02401323, 0.39497949, 0.44438914, 0.30156124, 0.56221215,
        0.2885587 , 0.13112795, 0.31856285, 0.76419796, 0.63973844,
        0.93242522, 0.98415072, 0.30010082, 0.77681532, 0.69921869,
        0.39928841, 0.9964777 , 0.0010418 , 0.99648271, 0.01693724,
        0.02954719, 0.01528119, 0.018352  , 0.01367551, 0.00978186]])

In [53]:
model.predict(features)

/uscms/home/srosenzw/nobackup/miniconda3/envs/work/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


array([[8.2483965e-08, 1.5719328e-06, 4.3848933e-05, 3.8734169e-07,
        1.9217238e-02, 1.5593958e-05, 9.8072124e-01]], dtype=float32)